In [2]:
import csv
import cv2
import numpy as np
import sklearn
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Conv2D
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [3]:
samples = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        samples.append(line)

train_samples, validation_samples = train_test_split(samples, test_size=0.2)


def generator(samples, batch_size):
    num_samples = len(samples)
    while True: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                
                steering_center = float(batch_sample[3])

                # create adjusted steering measurements for the side camera images
                correction = 0.2 # this is a parameter to tune
                steering_left = steering_center + correction
                steering_right = steering_center - correction

                # read in images from center, left and right cameras
                path = "./data/"

                img_center = cv2.imread(path + batch_sample[0])
                img_left = cv2.imread(path + batch_sample[1].split(' ')[-1])
                img_right = cv2.imread(path + batch_sample[2].split(' ')[-1])

                # add images and angles to data set
                images.append(img_center)
                angles.append(steering_center)
                images.append(img_left)
                angles.append(steering_left)
                images.append(img_right)
                angles.append(steering_right)
                
                # add augmented data
                images.append(cv2.flip(img_center, 1))
                angles.append(-steering_center)
                images.append(cv2.flip(img_left, 1))
                angles.append(-steering_left)
                images.append(cv2.flip(img_right, 1))
                angles.append(-steering_right)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

# The CNN has to predict the steering angle, a continuous number. It's a regression Network
model = Sequential()
model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Conv2D(24, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(36, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(48, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(64, (3,3),activation='relu'))
model.add(Conv2D(64, (3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dropout(0.3))
model.add(Dense(10))
model.add(Dropout(0.3))
model.add(Dense(1)) # I only want to predict steering angle

model.compile(loss='mse', optimizer='adam')
history_object = model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
                    validation_data=validation_generator, validation_steps=len(validation_samples), epochs=2, verbose = 1)

model.save('model_aug.h5')

Epoch 1/2
8131/8131 [==============================] - 16328s 2s/step - loss: 0.0195 - val_loss: 0.0229
Epoch 2/2
8131/8131 [==============================] - 16344s 2s/step - loss: 0.0099 - val_loss: 0.0303
